<a href="https://colab.research.google.com/github/Faryal112/Faryal112/blob/main/FYP_Model_04%3E%3ELSTM_%26_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report


In [3]:
from google.colab import files
uploaded = files.upload()


Saving data_stream-oper.csv to data_stream-oper.csv


In [5]:
# Load the dataset (update path if needed)
data = pd.read_csv('/content/data_stream-oper.csv')

# Drop columns with all NaN values
data = data.dropna(axis=1, how='all')

# Display the first few rows to understand the data
print(data.head())
print(data.info())


            valid_time  latitude  longitude  number  expver       u10  \
0  2020-05-01 00:00:00      36.0      70.83       0       1 -0.236581   
1  2020-05-01 00:00:00      36.0      71.08       0       1 -0.091348   
2  2020-05-01 00:00:00      36.0      71.33       0       1  0.447348   
3  2020-05-01 00:00:00      36.0      71.58       0       1  0.364432   
4  2020-05-01 00:00:00      36.0      71.83       0       1 -0.109628   

        v10        sp       hcc       lcc  ...   p90.162   p88.162  p89.162  \
0  0.635636  61000.68  0.998981  0.059074  ...  0.446365  0.000939  0.00145   
1  0.342240  59817.18  0.998553  0.007935  ...  0.402031  0.000275  0.00145   
2 -0.264296  61669.18  0.996925  0.009781  ...  0.397300  0.000275  0.00145   
3 -0.623305  65727.43  0.999996  0.003407  ...  0.465660  0.000275  0.00145   
4 -0.731063  67074.18  1.000000  0.000000  ...  0.548210  0.000275  0.00145   

       licd          lmld       ltlt        tsn  slt         vithe    lai_hv  
0  1.31

In [6]:
# For illustration: creating a random target column as 'cloud_burst'
# Replace this logic with real criteria if available
np.random.seed(42)
data['cloud_burst'] = np.random.randint(0, 2, data.shape[0])

# Drop irrelevant columns like 'valid_time' and any others not needed
data = data.drop(columns=['valid_time', 'sst'], errors='ignore')

# Define X and y
X = data.drop(columns=['cloud_burst'])
y = data['cloud_burst']


In [7]:
# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [8]:
# Scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [10]:
# Reshape input data to fit CNN-LSTM model (samples, time steps, features)
time_steps = 1  # Adjust as needed based on temporal resolution
# Calculate n_features such that the reshape operation is valid
n_features = X_train.shape[1]
# You can adjust time_steps to any factor of n_features.
# For e.g. time_steps=23 and n_features=1 will work

X_train = X_train.reshape((X_train.shape[0], time_steps, n_features))
X_test = X_test.reshape((X_test.shape[0], time_steps, n_features))

In [13]:
# Define CNN-LSTM model
model = Sequential([
    Conv1D(filters=64, kernel_size=1, activation='relu', input_shape=(time_steps, n_features)),
    MaxPooling1D(pool_size=1),
    # Remove Flatten() layer
    LSTM(50, return_sequences=True),  # This LSTM now receives 3D output from MaxPooling1D
    Dropout(0.2),
    LSTM(50),
    Dropout(0.2),
    Dense(50, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [14]:
# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)


Epoch 1/10
467/467 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - accuracy: 0.5083 - loss: 0.6933 - val_accuracy: 0.5040 - val_loss: 0.6931
Epoch 2/10
467/467 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.5100 - loss: 0.6931 - val_accuracy: 0.4981 - val_loss: 0.6936
Epoch 3/10
467/467 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.5145 - loss: 0.6929 - val_accuracy: 0.5094 - val_loss: 0.6933
Epoch 4/10
467/467 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.5132 - loss: 0.6927 - val_accuracy: 0.5032 - val_loss: 0.6935
Epoch 5/10
467/467 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.5166 - loss: 0.6924 - val_accuracy: 0.4981 - val_loss: 0.6941
Epoch 6/10
467/467 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.5225 - loss: 0.6917 - val_accuracy: 0.5037 - val_loss: 0.6942
Epoch 7/10
467/467 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.5188 - loss: 0.6919 - val_accuracy: 0.4957 - val_loss: 0.6956
Epoch 8/10
467/467 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.5252 - loss: 0.6903 - val_accuracy: 0

In [15]:
# Evaluate the model
y_pred = (model.predict(X_test) > 0.5).astype("int32")
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


146/146 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Accuracy: 0.4980728051391863
              precision    recall  f1-score   support

           0       0.49      0.63      0.55      2307
           1       0.51      0.37      0.43      2363

    accuracy                           0.50      4670
   macro avg       0.50      0.50      0.49      4670
weighted avg       0.50      0.50      0.49      4670



In [18]:
y.value_counts()

,count
cloud_burst,
1,11686
0,11664
